In [7]:
import pandas as pd
import numpy as np
from funcs import *
import unidecode
import basedosdados as bd

In [8]:
# 2.4. DETERMINANTE MERCADO

mercado = {}

In [9]:
# 2.4.1. Subdeterminante Desenvolvimento Econômico

subdet = 'Desenvolvimento Econômico'

In [10]:
# 2.4.1.1. Indicador Índice de Desenvolvimento Humano

ind = pd.read_excel('Arquivos ICE - 23/Ind_Originais_ICE_2022.xlsx', header=[i for i in range(6)], index_col=[0,1])
sub_desenveco = ind['Mercado', 'Desenvolvimento Econômico', 'Índice de Desenvolvimento Humano']
sub_desenveco.columns = sub_desenveco.columns.droplevel([0,1])
sub_desenveco.columns.values[0] = 'Índice de Desenvolvimento Humano'
sub_desenveco = sub_desenveco.rename_axis(None, axis=1)
sub_desenveco.index.names = ['Município', 'UF']

In [11]:
# 2.4.1.2. Indicador Crescimento Médio Real do PIB

pib = pd.read_excel('DETERMINANTE MERCADO/tabela5938.xlsx', header=3).head(-1)
pib.columns.values[0] = 'Município'
pib['UF'] = pib['Município'].apply(lambda x: x.split('(')[-1][:2])
pib['Município'] = pib['Município'].apply(lambda x: x.split('(')[0].strip())
pib = pib.set_index(['Município', 'UF'])

# O DEFLATOR DO PIB UTILIZADO É DA FONTE IPEADATA E NÃO IBGE

deflator = pd.DataFrame([{
    '2016':8.10,
    '2017':3.67,
    '2018':4.49,
    '2019':4.22
}]).T
deflator = (deflator +100)/(deflator.loc['2019',0]+100)

pib_def = pd.DataFrame()
for i in pib.columns:
    pib_def[i] = pib[i]*deflator.loc[i,0]
    
pib_var = (pib_def.T / pib_def.T.shift(1)).apply(lambda x: x-1).T.drop('2016', axis=1)
pib_var['Crescimento Real Médio do PIB'] = pib_var.mean(axis=1)

sub_desenveco = pd.merge(sub_desenveco, pib_var['Crescimento Real Médio do PIB'], left_index=True, right_index=True)

In [12]:
# 2.4.1.3. Indicador Número de Empresas Exportadoras com Sede na Cidade

emp_exp = pd.read_excel('DETERMINANTE MERCADO/EMPRESAS_CADASTRO_2020.xlsx', header=7)

convert = lambda x: unidecode.unidecode(x.upper())
n_exp = {n:len(emp_exp.groupby(['MUNICÍPIO', 'UF']).get_group(tuple([convert(i) for i in n]))) for n in sub_desenveco.index}
n_exp = pd.DataFrame([n_exp],index=['n_exp']).T
n_exp.index = pd.MultiIndex.from_tuples(n_exp.index, names=['Município', 'UF'])

variaveis = ('COUNT(quantidade_vinculos_ativos), id_municipio')

## Montando a query
base = '`basedosdados.br_me_rais.microdados_estabelecimentos`'
project_id = "trim-descent-346220"
query = f"SELECT id_municipio, COUNT(quantidade_vinculos_ativos) FROM {base} WHERE ano = 2020 AND quantidade_vinculos_ativos > 0 GROUP BY id_municipio"

## Importando o data lake
df_rais = bd.read_sql(query=query, billing_project_id=project_id).set_index('id_municipio')

cod = pd.read_excel('DETERMINANTE MERCADO/RELATORIO_DTB_BRASIL_MUNICIPIO.xls')
cod = cod.rename(columns={'Código Município Completo':'id_municipio'})
cod['id_municipio'] = cod['id_municipio'].apply(str)
cod = cod[['Nome_Município','Nome_UF','id_municipio']].set_index('id_municipio')

n_rais = pd.merge(cod, df_rais, left_index=True, right_index=True).rename(columns={
    'Nome_Município':'Município',
    'Nome_UF':'UF',
    'f0_':'n_rais'
}).set_index(['Município', 'UF'])

ratio = pd.merge(n_rais, n_exp, left_index=True, right_index=True)
ratio['ratio'] = ratio['n_exp']/ratio['n_rais']

sub_desenveco['Número de Empresas Exportadoras com Sede na Cidade'] = ratio['ratio']

Downloading: 100%|████████████████████████████████████████████████████████████| 5570/5570 [00:00<00:00, 10198.94rows/s]


In [13]:
missing_data(sub_desenveco)
#extreme_values(sub_desenveco)
create_subindex(sub_desenveco, subdet)
mercado[subdet] = sub_desenveco
sub_desenveco

,,Índice de Desenvolvimento Humano,Crescimento Real Médio do PIB,Número de Empresas Exportadoras com Sede na Cidade,Índice de Desenvolvimento Econômico
Município,UF,,,,
Rio de Janeiro,RJ,0.799,0.014996,0.005925,5.873209
São Paulo,SP,0.805,0.025963,0.013096,6.554610
Florianópolis,SC,0.847,0.043613,0.005943,6.916514
Curitiba,PR,0.823,0.035563,0.010232,6.751256
Joinville,SC,0.809,0.097477,0.017927,7.927349
...,...,...,...,...,...
São Vicente,SP,0.768,0.018591,0.004187,5.406877
São João de Meriti,RJ,0.719,0.002492,0.001978,4.392700
Ribeirão das Neves,MG,0.684,0.050157,0.003631,4.712603


In [48]:
# 2.4.2. Subdeterminante Clientes Potenciais

subdet = 'Clientes Potenciais'
sub_clipot = pd.DataFrame()

In [49]:
amostra = pd.read_csv('AMOSTRA/100-municipios.csv').rename(columns={
    'COD. MUNIC':'Município'
}).rename(columns={'Município':'None', 'NOME DO MUNICÍPIO':'Município'}).set_index(['Município', 'UF'])

sub_clipot['PIB per capita'] = (pib['2019']/amostra['POPULAÇÃO ESTIMADA']).dropna()
sub_clipot

,,PIB per capita
Município,UF,
Ananindeua,PA,15.110366
Anápolis,GO,37.168564
Aparecida de Goiânia,GO,23.877938
Aracaju,SE,26.069206
Bauru,SP,40.147629
...,...,...
Uberlândia,MG,53.252704
Vila Velha,ES,25.281651
Vitória,ES,58.492298
